In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

train_data_path = '/Users/bahar/Downloads/heart_train.csv'  
train_data = pd.read_csv(train_data_path)
type(train_data)

pandas.core.frame.DataFrame

In [2]:
# train_data = train_data.iloc[:,:].values

In [3]:
# train_data

In [4]:
categorical_cols = train_data.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']  


numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X = train_data.drop('HeartDisease', axis=1)
y = train_data['HeartDisease']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

model =  LogisticRegression(max_iter=1000, random_state=0)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_valid)
accuracy = accuracy_score(y_valid, preds)
print(accuracy)

test_data_path = '/Users/bahar/Downloads/heart_test.csv'  
test_data = pd.read_csv(test_data_path)
X_test = test_data.drop(['Unnamed: 0', 'id'], axis=1)  
X_test_preprocessed = preprocessor.transform(X_test)

logistic_regression_predictions = model.predict(X_test_preprocessed)

0.8682170542635659


In [5]:
predictions_df = pd.DataFrame({
    "id": test_data['id'],
    "output": logistic_regression_predictions
})
output_path = '/Users/bahar/Desktop/heart_disease_predictions-LR.csv'

predictions_df.to_csv(output_path, index=False)